# Regression with an Abalone Dataset
Run after following notebooks are run:
1. **01_Data_Cleaning**
2. **02_EDA**

## Development Notes/Ideas
- rf
- xgb
- knn
- lasso
- stack


## Libraries

In [1]:
## set up libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

## Load Data

In [2]:
train = pd.read_pickle('train_clean.pkl')
test = pd.read_pickle('test_clean.pkl')

## separate in to features and response variable
x_train = train.drop('Rings', axis=1)
y_train = train['Rings']

x_test = test

## One Hot Encoding

In [3]:
x_train_onehot = pd.get_dummies(x_train, drop_first=True)
x_test_onehot = pd.get_dummies(x_test, drop_first=True)
print("train dimensions: ", x_train_onehot.shape, "\n","test dimensions: ", x_test_onehot.shape)
x_train_onehot.columns.to_list()

train dimensions:  (90615, 11) 
 test dimensions:  (90615, 11)


['Length',
 'Diameter',
 'Height',
 'Whole weight',
 'Whole weight.1',
 'Whole weight.2',
 'Shell weight',
 'Volume_cube',
 'Density_meat',
 'Sex_I',
 'Sex_M']

## Modelling
### Random Forest

In [30]:
## hyperparameter tuning: n_estimators
param_grid = {'n_estimators' : np.arange(300,450,50), 'n_jobs' : [-1]}
kf = KFold(n_splits=5, shuffle=True, random_state=888)

rf_tuning = GridSearchCV(RandomForestRegressor(random_state=888), param_grid, scoring='neg_mean_squared_log_error', cv=kf, n_jobs=-1)
rf_tuning.fit(x_train_onehot.values, y_train)
None

In [31]:
## find best param with highest accuracy
rf_tuning_accr = rf_tuning.cv_results_['mean_test_score']
print('best_params =', rf_tuning.best_params_)
print('best_score =', rf_tuning.best_score_)
rf_tuning_accr

best_params = {'n_estimators': 400, 'n_jobs': -1}
best_score = -0.02387037717728524


array([-0.02389283, -0.02387928, -0.02387038])

In [32]:
## instantiate rf algorithm, using best tuned param
rf = RandomForestRegressor(n_estimators=400, random_state=888, n_jobs=-1)

## fit model to training data
rf.fit(x_train_onehot.values, y_train); None

In [33]:
## List feature importance
rf_ft_imp = pd.DataFrame(data={'Features': x_train_onehot.columns, 'Gini Importance':rf.feature_importances_*100})
rf_ft_imp.sort_values(by='Gini Importance', ascending=False)

,Features,Gini Importance
6,Shell weight,53.240623
8,Density_meat,13.037074
4,Whole weight.1,7.862940
3,Whole weight,6.250325
5,Whole weight.2,5.553472
7,Volume_cube,3.918834
0,Length,3.877678
1,Diameter,3.000796
2,Height,1.926414
9,Sex_I,0.760278
